Ejercicio 2:

Script 1: Grabación del dataset (record-dataset.py)
Usar la cámara web para capturar imágenes de la mano.
Utilizar MediaPipe para detectar los landmarks de la mano (21 puntos clave con coordenadas x y y).
Almacenar las coordenadas de los landmarks junto con la etiqueta correspondiente (0 para "piedra", 1 para "papel", 2 para "tijeras") en archivos .npy (por ejemplo, rps_dataset.npy y rps_labels.npy).

In [3]:
import cv2
import numpy as np
import mediapipe as mp
import os

In [4]:
# Inicializamos Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

In [5]:
# Cargamos datasets si existen
try:
    rps_dataset = list(np.load('rps_dataset.npy', allow_pickle=True))
    rps_labels = list(np.load('rps_labels.npy', allow_pickle=True))
    print("Dataset cargado exitosamente.")
except FileNotFoundError:
    print("No se encontró un dataset existente. Se creará uno nuevo.")
    rps_dataset = []
    rps_labels = []

Dataset cargado exitosamente.


In [6]:
# Definimos la carpeta de destino
save_dir = r'C:\\Users\\valen\\Downloads\\Parte 2 TP'

# Creamos subcarpetas para cada clase si no existen
gestures = ['piedra', 'papel', 'tijeras']
for gesture in gestures:
    os.makedirs(os.path.join(save_dir, gesture), exist_ok=True)

In [8]:
# Función para capturar gestos
def capture_gestures():
    cap = cv2.VideoCapture(0)
    labels = {0: 'piedra', 1: 'papel', 2: 'tijeras'}
    
    while cap.isOpened():
        print("Capturando gestos. Presiona 'q' para tomar una foto de cada gesto (piedra, papel, tijeras), y 'p' para detener.")
        
        for label, gesture_name in labels.items():
            print(f"Preparado para capturar gesto {gesture_name}. Presiona 'q' para tomar una foto.")
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = hands.process(rgb_frame)
                
                if result.multi_hand_landmarks:
                    for hand_landmarks in result.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        
                        # Obtenemos coordenadas de los landmarks de la mano
                        landmarks = []
                        for lm in hand_landmarks.landmark:
                            landmarks.append([lm.x, lm.y])
                
                # Mostramos la captura
                cv2.imshow('Hand Gesture Capture', frame)
                
                # Presionando "q" se captura el gesto
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    if result.multi_hand_landmarks:
                        # Guardamos la imagen del gesto en la carpeta específica
                        img_name = f"{gesture_name}_{len(os.listdir(os.path.join(save_dir, gesture_name)))}.png"
                        save_path = os.path.join(save_dir, gesture_name, img_name)
                        cv2.imwrite(save_path, frame)

                        rps_dataset.append(landmarks)
                        rps_labels.append(label)
                        print(f"Foto del gesto {gesture_name} capturada y guardada en {save_path}.")
                    break
                
                # Presionando "p" se detiene el proceso de captura
                if cv2.waitKey(10) & 0xFF == ord('p'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return
    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# Ejecutamos el ciclo de captura
capture_gestures()

# Guardamos el dataset actualizado
np.save('rps_dataset.npy', np.array(rps_dataset))
np.save('rps_labels.npy', np.array(rps_labels))
print("Dataset actualizado y guardado exitosamente.")

Capturando gestos. Presiona 'q' para tomar una foto de cada gesto (piedra, papel, tijeras), y 'p' para detener.
Preparado para capturar gesto piedra. Presiona 'q' para tomar una foto.
Dataset actualizado y guardado exitosamente.


Script 2
Entrenamos un modelo con el dataset generado para predecir si en una imagen se ve piedra, papel o tijera.

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Cargamos los datos del dataset
rps_dataset = np.load('rps_dataset.npy', allow_pickle=True)
rps_labels = np.load('rps_labels.npy', allow_pickle=True)

# Convertimos la lista de landmarks en un array NumPy y ajustar el tamaño adecuado
rps_dataset = np.array(rps_dataset)
rps_dataset = rps_dataset.reshape(rps_dataset.shape[0], 42)  # 21 puntos clave (x, y) = 42 entradas

# Normalizamos los datos
rps_dataset = rps_dataset / np.max(rps_dataset)

# Convertimos etiquetas a formato one-hot encoding
label_binarizer = LabelBinarizer()
rps_labels = label_binarizer.fit_transform(rps_labels)

# Dividimos los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(rps_dataset, rps_labels, test_size=0.2, random_state=42)

# Creamos el modelo de la red neuronal con capas densas
model = Sequential([
    Dense(64, activation='relu', input_shape=(42,)),  # 42 entradas (21 puntos clave x, y)
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # 3 salidas (piedra, papel, tijeras)
])



c:\Users\valen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:

# Compilamos el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall(name='recall')])

# Entrenamos el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Guardamos el modelo entrenado
model.save('rps_model.h5')

print("Modelo entrenado y guardado como rps_model.h5")

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 279ms/step - accuracy: 0.3564 - loss: 1.0874 - recall: 0.0000e+00 - val_accuracy: 0.3000 - val_loss: 1.0976 - val_recall: 0.0000e+00
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3773 - loss: 1.0503 - recall: 0.0000e+00 - val_accuracy: 0.3000 - val_loss: 1.0970 - val_recall: 0.0000e+00
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3877 - loss: 1.0276 - recall: 0.0000e+00 - val_accuracy: 0.3000 - val_loss: 1.0725 - val_recall: 0.0000e+00
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3668 - loss: 1.0140 - recall: 0.0000e+00 - val_accuracy: 0.3000 - val_loss: 1.0422 - val_recall: 0.0000e+00
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6692 - loss: 0.9928 - recall: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 1.0269 - val_recall: 0.0000e+00
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6692 - loss: 0.9703 - recall: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 1.016

Modelo entrenado y guardado como rps_model.h5


Script 3

Utilizando el modelo predecimos en tiempo real los gestos que haga una mano.

In [10]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Inicializamos MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Cargamos el modelo entrenado
model = load_model('rps_model.h5')

# Etiquetas de los gestos
gestures = ["Piedra", "Papel", "Tijeras"]

# Captura de la cámara
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertimos la imagen a RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Detección de los landmarks de la mano
    results = hands.process(image_rgb)

    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Extraemos las coordenadas x e y de los landmarks
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append(landmark.x)
                landmarks.append(landmark.y)
            
            # Convertimos a formato numpy array y hacemos la predicción
            landmarks = np.array(landmarks).reshape(1, -1)
            prediction = model.predict(landmarks)
            gesture = gestures[np.argmax(prediction)]

            # Dibujamos los landmarks en la imagen
            mp_drawing.draw_landmarks(image_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Mostramos el gesto reconocido en pantalla
            cv2.putText(image_bgr, f'Gesto: {gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Mostramos la imagen en pantalla
    cv2.imshow('Piedra, Papel o Tijeras', image_bgr)

    # Salimos del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()
hands.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

Conclusion

En este ejercicio creamos satisfactoriamente un script que nos permitio generar rapidamente un dataset modesto con imagenes de piedra, papel o tijera, para luego entrenar un modelo con ese dataset que finalmente utilizamos para predecir en tiempo real que gesto esta haciendo una mano y escribir en pantalla la prediccion.
Al ser una tarea sencilla para alcanzar una muy buena accuracy no hizo falta utilizar un modelo complejo, se utilizaron capas densas.
 